<a href="https://colab.research.google.com/github/prateekrana17/SPARK-NLP-RAG-LLM/blob/main/SPARK-NLP-latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh


--2024-05-14 10:09:57--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘colab_setup.sh’

colab_setup.sh      100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-05-14 10:09:57 (45.1 MB/s) - ‘colab_setup.sh’ saved [1191/1191]



In [2]:
!bash colab_setup.sh

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 12.8 MB/s eta 0:00:00


In [12]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pyspark.sql.functions as F




In [14]:
spark = sparknlp.start()
spark

In [5]:
diff_lang_text_list = [
    "This is an English text.",
    "Este es un texto en español.",
    "Ceci est un texte en français.",
    "Dies ist ein deutscher Text."
]
data = spark.createDataFrame(diff_lang_text_list, StringType()).toDF("text")


In [6]:
data.show()

+--------------------+
|                text|
+--------------------+
|This is an Englis...|
|Este es un texto ...|
|Ceci est un texte...|
|Dies ist ein deut...|
+--------------------+



In [7]:
# Language Detection Pipeline
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

language_detector = LanguageDetectorDL.pretrained() \
    .setInputCols(["document"]) \
    .setOutputCol("language")

language_detection_pipeline = Pipeline(stages=[document_assembler, language_detector])


ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
[OK!]


In [15]:
# function for translation pipeline

def get_translation_pipeline(source_lang, target_lang):
    model_name = f"opus_mt_{source_lang}_{target_lang}"

    sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx") \
        .setInputCols(["document"]) \
        .setOutputCol("sentence")

    marian = MarianTransformer.pretrained(model_name) \
        .setInputCols(["sentence"]) \
        .setOutputCol("translation")

    return Pipeline(stages=[document_assembler, sentence_detector, marian])


In [9]:
# translate text based on language
def translate_text(text, source_lang, target_lang):
    translation_pipeline = get_translation_pipeline(source_lang, target_lang)
    single_text_df = spark.createDataFrame([[text]], ["text"])
    result = translation_pipeline.fit(single_text_df).transform(single_text_df)
    return result.selectExpr("explode(translation.result) as translation").collect()[0][0]

translate_udf = udf(translate_text, StringType())


In [16]:
target_lang = "en"  # Example target language

detected_lang_df = language_detection_pipeline.fit(data).transform(data)
detected_lang_df = detected_lang_df.withColumn("detected_language", detected_lang_df.language.result[0])

translated_df = detected_lang_df.withColumn("translation", translate_udf(detected_lang_df.text, detected_lang_df.detected_language, F.lit(target_lang)))

translated_df.select("text", "detected_language", "translation").show(truncate=False)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 563, in dump
    return Pickler.dump(self, obj)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/context.py", line 361, in __getnewargs__
    raise RuntimeError(
RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.


PicklingError: Could not serialize object: RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.